### 1. Settings

In [1]:
#####################################
##########  DEPENDECIES ############
#####################################

import os
import pickle
import sys
sys.path.append("../")

from tqdm import tqdm # type: ignore
from datetime import date

import evaluate

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import utils.prep as pr
import utils.eval as ev
import utils.inference as infer
from utils.sampling import create_splits, prep_cv_validation
from utils.training import cv_cluster_set, cv_training_epochs_sets, test_cluster_set
from utils.training import results_dict_todf, cv_step_2, full_step_2, test_training_epochs_sets
from utils.inference import meta_predict, create_ensemble_map, ensemble_compute

tqdm.pandas()
import warnings
warnings.filterwarnings("ignore")

os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["WANDB_DISABLED"] = "true"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import torch

#####################################
############  CONSTANTS #############
#####################################

RS = 42
BATCH_SIZE = 16
DECODER_LENGTH = 25
ENCODER_LENGTH = 25
MODEL_NAME = "Salesforce/codet5-base-multi-sum"

FULL_TRAIN_ARGS = {
    "BATCH_SIZE": BATCH_SIZE,
    "DECODER_LENGTH": DECODER_LENGTH,
    "ENCODER_LENGTH": ENCODER_LENGTH,
    "SEQ_TRAINER_ARGS": {
        "overwrite_output_dir": True,
        "num_train_epochs": [0, 1, 2, 5, 10],
        "do_train": True,
        "do_eval": True,
        "per_device_train_batch_size": 4,
        "per_device_eval_batch_size": 4,
        "learning_rate": 1e-5,
        "warmup_steps": 500,
        "weight_decay": 0.1,
        "label_smoothing_factor": 0.1,
        "predict_with_generate": True,
        "logging_steps": 100,
        "save_total_limit": 1,
        "save_strategy": "no",
        "logging_strategy": "epoch",
        "evaluation_strategy": "epoch",
        "load_best_model_at_end": False,
        "output_dir" : 'reports/results',
        "logging_dir" : "reports/logs",
    },
}

experiment_config = {
    "DATA_STR" : "20240721",
    "RS" : RS,
    "DRIFT_TYPE" : "no_drift",
    "NFOLD" : 3,
    "FULL_TRAIN_ARGS" : FULL_TRAIN_ARGS,
    "MODEL_NAME" : MODEL_NAME,
    "CLUSTER_EPOCHS" : 5,
    "CLUSTER_SET_ID" : [1, 4, 3, [0, 1, 4,]],
    "TRAIN_SIZE" : 7000,
    "TEST_SIZE" : 2500,
}
experiment_config["ANALYSIS_POSTFIX"] = f"mined_{experiment_config['DRIFT_TYPE']}_{str(date.today())}"
experiment_config["FULL_TRAIN_ARGS"]["SEQ_TRAINER_ARGS"]["output_dir"] += "/" + experiment_config["ANALYSIS_POSTFIX"] 
experiment_config["FULL_TRAIN_ARGS"]["SEQ_TRAINER_ARGS"]["logging_dir"] += "/" + experiment_config["ANALYSIS_POSTFIX"] 

if not os.path.exists(experiment_config["FULL_TRAIN_ARGS"]["SEQ_TRAINER_ARGS"]["logging_dir"]):
    os.mkdir(experiment_config["FULL_TRAIN_ARGS"]["SEQ_TRAINER_ARGS"]["logging_dir"])

if not os.path.exists(experiment_config["FULL_TRAIN_ARGS"]["SEQ_TRAINER_ARGS"]["output_dir"]):
    os.mkdir(experiment_config["FULL_TRAIN_ARGS"]["SEQ_TRAINER_ARGS"]["output_dir"])

tokenizer = AutoTokenizer.from_pretrained(experiment_config["MODEL_NAME"], skip_special_tokens=False)
model = AutoModelForSeq2SeqLM.from_pretrained(experiment_config["MODEL_NAME"])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
rouge = evaluate.load('rouge')

[nltk_data] Downloading package punkt to
[nltk_data]     /home/RDC/zinovyee.hub/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### 2. Conala data. Preprocessing. 

In [2]:
sampling_dict = create_splits(experiment_config=experiment_config, tokenizer=tokenizer, train_size=experiment_config["TRAIN_SIZE"], test_size=experiment_config["TEST_SIZE"], cluster_id=4)
train_dataset, test_data, test_df, train_df = sampling_dict["train_data"], sampling_dict["test_data"], sampling_dict["test_df"], sampling_dict["train_df"]

splits, questions_list = prep_cv_validation(train_dataset=train_dataset, 
                            experiment_config=experiment_config)

Train Data:  (7000, 11)
Test Data:  (2500, 11)
Train Data: Cluster cluster
2    3888
3    2114
4     596
1     234
0     168
Name: count, dtype: int64
Test Data: Cluster cluster
2    1358
3     777
4     220
1      74
0      71
Name: count, dtype: int64


Filter:   0%|          | 0/7000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/7000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2500 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2500 [00:00<?, ? examples/s]

Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

Fold 0
Fold 1
Fold 2


In [3]:
fold_results = cv_training_epochs_sets(experiment_config=experiment_config,
                            splits=splits,
                            questions_list=questions_list,
                            train_dataset=train_dataset,
                            tokenizer=tokenizer)

with open(f'reports/results/{experiment_config["ANALYSIS_POSTFIX"]}/cv_fold_epoch_set.pickle', 'wb') as handle:
    pickle.dump(fold_results, handle, protocol=pickle.HIGHEST_PROTOCOL)

Fold 0


Filter:   0%|          | 0/7000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/7000 [00:00<?, ? examples/s]

TRAINING EPOCH SET 0
TRAINING EPOCHS 0
LOADING MODEL Salesforce/codet5-base-multi-sum
cuda


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


...

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len,Bleu,Brevity Penalty,Length Ratio,Translation Length,Reference Length
1,4.264500,4.498985,0.277900,0.078000,0.245100,0.245300,13.712000,0.029000,1.000000,1.101100,24824,22545
2,4.404900,4.458007,0.280800,0.077500,0.245500,0.245500,13.718800,0.027900,1.000000,1.104100,24892,22545
3,4.294800,4.456551,0.282400,0.077900,0.247600,0.247500,13.747100,0.028100,1.000000,1.099800,24794,22545


TRAINING EPOCH SET 10
TRAINING EPOCHS 5
LOADING MODEL ./tmp/
cuda


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len,Bleu,Brevity Penalty,Length Ratio,Translation Length,Reference Length
1,3.797400,4.505895,0.276900,0.074800,0.243700,0.243900,13.441100,0.027800,1.000000,1.067400,24064,22545
2,4.007000,4.479992,0.276500,0.072700,0.240800,0.241000,13.533200,0.027500,1.000000,1.081600,24385,22545
3,4.026000,4.457977,0.279500,0.076200,0.245500,0.246000,13.667400,0.028500,1.000000,1.087200,24511,22545
4,4.018300,4.457972,0.281300,0.077300,0.247900,0.248300,13.691000,0.029600,1.000000,1.084700,24454,22545
5,3.970600,4.463252,0.278800,0.074800,0.245500,0.245700,13.610400,0.028800,1.000000,1.079800,24344,22545


In [7]:
with open(f'reports/results/{experiment_config["ANALYSIS_POSTFIX"]}/cv_fold_epoch_set.pickle', 'rb') as handle:
   fold_results = pickle.load(handle)
   
for cluster_idx in experiment_config["CLUSTER_SET_ID"]:
    fold_results = cv_cluster_set(experiment_config=experiment_config,
                                            splits=splits,
                                            questions_list=questions_list,
                                            train_dataset=train_dataset,
                                            tokenizer=tokenizer,
                                            fold_results=fold_results,
                                            cluster_id=cluster_idx)

cv_df = results_dict_todf(fold_results)

########## SAVE THE FILE

with open(f'reports/results/{experiment_config["ANALYSIS_POSTFIX"]}/cv_step1.pickle', 'wb') as handle:
    pickle.dump(cv_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

Fold 0


Filter:   0%|          | 0/7000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/7000 [00:00<?, ? examples/s]

Cluster [1] training size (160, 14)
TRAINING CLUSTER SET [1] FOR EPOCHS 5
LOADING MODEL Salesforce/codet5-base-multi-sum
cuda


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


...

Fold 2


Filter:   0%|          | 0/7000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/7000 [00:00<?, ? examples/s]

Cluster [0, 1, 4] training size (670, 14)
TRAINING CLUSTER SET [0, 1, 4] FOR EPOCHS 5
LOADING MODEL ./tmp/
cuda


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [8]:
print("Mean")
print(cv_df.groupby(["model_set"])["rouge"].mean())

print("STD")
print(cv_df.groupby("model_set")["rouge"].std())

Mean
model_set
0                    0.128471
1                    0.272787
2                    0.280769
5                    0.285255
10                   0.282760
cluster_[0, 1, 4]    0.241250
cluster_[1]          0.141068
cluster_[3]          0.259440
cluster_[4]          0.213261
Name: rouge, dtype: float64
STD
model_set
0                    0.135276
1                    0.159897
2                    0.161486
5                    0.162626
10                   0.163305
cluster_[0, 1, 4]    0.161949
cluster_[1]          0.140849
cluster_[3]          0.167286
cluster_[4]          0.159754
Name: rouge, dtype: float64


### Step 2. Learn performance

In [3]:
########## LOAD CV RESULTS

import pickle
with open(f'reports/results/{experiment_config["ANALYSIS_POSTFIX"]}/cv_step1.pickle', 'rb') as handle:
    cv_df = pickle.load(handle)

########## RUN STEP 2 ON CV

cv_df, model_results = cv_step_2(experiment_config=experiment_config, cv_df=cv_df, t_models=["lgbm"])

with open(f'reports/results/{experiment_config["ANALYSIS_POSTFIX"]}/s2_model_results.pickle', 'wb') as handle:
    pickle.dump(model_results, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(f'reports/results/{experiment_config["ANALYSIS_POSTFIX"]}/cv_results.pickle', 'wb') as handle:
    pickle.dump(cv_df, handle, protocol=pickle.HIGHEST_PROTOCOL)


0
lgbm
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 8.063856 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 13044
[LightGBM] [Info] Number of data points in the train set: 41994, number of used features: 1032
[LightGBM] [Info] Start training from score 0.232153
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No 

In [4]:
### TO SAVE THE VECTORIZER AND STEP 2 MODELS

with open(f'reports/results/{experiment_config["ANALYSIS_POSTFIX"]}/cv_results.pickle', 'rb') as handle:
    cv_df = pickle.load(handle)


print("Mean")
print(cv_df.groupby(["model_set"])["rouge"].mean())

print("STD")
print(cv_df.groupby("model_set")["rouge"].std())


full_step_2(cv_df=cv_df, 
            experiment_config=experiment_config, t_models=["lgbm"])

Mean
model_set
0                    0.128471
1                    0.272787
2                    0.280769
5                    0.285255
10                   0.282760
cluster_[0, 1, 4]    0.241250
cluster_[1]          0.141068
cluster_[3]          0.259440
cluster_[4]          0.213261
ensemble             0.284587
Name: rouge, dtype: float64
STD
model_set
0                    0.135276
1                    0.159897
2                    0.161486
5                    0.162626
10                   0.163305
cluster_[0, 1, 4]    0.161949
cluster_[1]          0.140849
cluster_[3]          0.167286
cluster_[4]          0.159754
ensemble             0.162828
Name: rouge, dtype: float64
lgbm
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing row-wise multi-thr

# TEST

In [5]:
# sampling_dict = create_splits(experiment_config=experiment_config, tokenizer=tokenizer, test=True, train_size=100, test_size=25, cluster_id=4)
# train_dataset, test_data, test_df, train_df = sampling_dict["train_data"], sampling_dict["test_data"], sampling_dict["test_df"], sampling_dict["train_df"]

# splits, questions_list = prep_cv_validation(train_dataset=train_dataset, 
#                             experiment_config=experiment_config)

In [6]:
with open(f"reports/results/{experiment_config['ANALYSIS_POSTFIX']}/cv_results.pickle", "rb") as handle:
    cv_resutls = pickle.load(handle)

base_models_list = list(cv_resutls.model_set.unique())
base_models_list.pop(-1)

'ensemble'

In [7]:
meta_preds_df = meta_predict(experiment_config=experiment_config, 
                    test_df=test_df,
                    base_models_names=base_models_list,
                    t_models=["lgbm"])

########## SAVE THE FILE

with open(f'reports/results/{experiment_config["ANALYSIS_POSTFIX"]}/test_step2.pickle', 'wb') as handle:
    pickle.dump(meta_preds_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

lgbm
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


In [10]:
with open(f'reports/results/{experiment_config["ANALYSIS_POSTFIX"]}/test_step2.pickle', 'rb') as handle:
    meta_preds_df = pickle.load(handle)
    
meta_preds_df.groupby("model_set").lgbm_preds.mean()

model_set
0                    0.136463
1                    0.271276
2                    0.277438
5                    0.280957
10                   0.278436
cluster_[0, 1, 4]    0.247229
cluster_[1]          0.148851
cluster_[3]          0.262917
cluster_[4]          0.220177
Name: lgbm_preds, dtype: float64

In [11]:
optimal_ensemble_map, ensemble_val_estim = create_ensemble_map(meta_preds_df=meta_preds_df, 
                                                                t_model_name="lgbm")

In [12]:
test_result_df = test_training_epochs_sets(experiment_config=experiment_config,
                            test_df=test_df,
                            test_data=test_data,
                            train_data=train_dataset,
                            tokenizer=tokenizer)

with open(f'reports/results/{experiment_config["ANALYSIS_POSTFIX"]}/test_epoch_set.pickle', 'wb') as handle:
    pickle.dump(test_result_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

TRAINING EPOCH SET 0
TRAINING EPOCHS 0
cuda


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


TRAINING EPOCH SET 1
TRAINING EPOCHS 1
cuda


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len,Bleu,Brevity Penalty,Length Ratio,Translation Length,Reference Length
1,4.961300,4.534938,0.275100,0.075900,0.243400,0.243700,13.992800,0.028700,1.000000,1.144900,27495,24015


...

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len,Bleu,Brevity Penalty,Length Ratio,Translation Length,Reference Length
1,4.231900,4.409790,0.286000,0.079000,0.249700,0.250000,13.817200,0.029400,1.000000,1.128000,27089,24015
2,4.357300,4.379532,0.284600,0.079700,0.249000,0.249000,13.762400,0.030700,1.000000,1.108600,26623,24015
3,4.266100,4.378836,0.285400,0.078600,0.249200,0.249300,13.896800,0.030100,1.000000,1.118700,26865,24015


TRAINING EPOCH SET 10
TRAINING EPOCHS 5
cuda


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len,Bleu,Brevity Penalty,Length Ratio,Translation Length,Reference Length
1,3.769400,4.415681,0.280600,0.074500,0.244100,0.244200,13.644000,0.028000,1.000000,1.103900,26510,24015
2,3.987500,4.397045,0.285000,0.079400,0.249200,0.249300,13.602400,0.030900,1.000000,1.091700,26218,24015
3,4.011700,4.377162,0.282700,0.077300,0.247900,0.247800,13.890800,0.029900,1.000000,1.115800,26796,24015
4,3.996300,4.380507,0.286700,0.077200,0.249900,0.250000,13.526000,0.028900,1.000000,1.081200,25966,24015
5,3.953200,4.387558,0.286800,0.077800,0.250300,0.250300,13.714400,0.028700,1.000000,1.096900,26341,24015


In [13]:
train_df.cluster.value_counts()

cluster
2    3888
3    2114
4     596
1     234
0     168
Name: count, dtype: int64

In [14]:
with open(f'reports/results/{experiment_config["ANALYSIS_POSTFIX"]}/test_epoch_set.pickle', 'rb') as handle:
   test_result_df = pickle.load(handle)

test_result_df = test_result_df.rename(columns={"epoch_set": "model_set"})

for cluster_idx in experiment_config["CLUSTER_SET_ID"]:
    test_result_df = test_cluster_set(experiment_config=experiment_config,
                                    test_df=test_df,
                                    test_data=test_data,
                                    tokenizer=tokenizer,
                                    results_df=test_result_df,
                                    cluster_id=cluster_idx,
                                    train_df=train_df)

########## SAVE THE FILE

with open(f'reports/results/{experiment_config["ANALYSIS_POSTFIX"]}/test_results.pickle', 'wb') as handle:
    pickle.dump(test_result_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

Cluster [1] training size (234, 14)
TRAINING CLUSTER SET [1] FOR EPOCHS5
LOADING MODEL Salesforce/codet5-base-multi-sum
cuda


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len,Bleu,Brevity Penalty,Length Ratio,Translation Length,Reference Length
1,6.069100,5.868877,0.126100,0.016000,0.114500,0.114700,9.592000,0.005100,0.712900,0.747100,17942,24015
2,5.892700,5.698154,0.128600,0.017000,0.117000,0.117100,9.544000,0.005500,0.706100,0.741900,17816,24015
3,5.664800,5.505741,0.132900,0.018900,0.120800,0.121000,9.454000,0.006300,0.694900,0.733200,17607,24015
4,5.391800,5.328917,0.142200,0.022600,0.128900,0.129300,9.387600,0.008100,0.681400,0.722700,17356,24015
5,5.185400,5.185640,0.158500,0.027100,0.143500,0.143800,9.678400,0.010500,0.717700,0.750900,18034,24015


Cluster [4] training size (596, 14)
TRAINING CLUSTER SET [4] FOR EPOCHS5
LOADING MODEL Salesforce/codet5-base-multi-sum
cuda


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


...

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len,Bleu,Brevity Penalty,Length Ratio,Translation Length,Reference Length
1,5.275900,4.786207,0.228100,0.055500,0.205800,0.205900,12.390000,0.023000,0.994800,0.994800,23890,24015
2,4.599700,4.616581,0.252900,0.065400,0.224400,0.224400,12.977600,0.026200,1.000000,1.056200,25364,24015
3,4.392300,4.577627,0.261000,0.068900,0.229500,0.229500,13.277200,0.027100,1.000000,1.081300,25968,24015
4,4.276300,4.557720,0.265100,0.070800,0.232900,0.233100,13.334800,0.027800,1.000000,1.085000,26056,24015
5,4.214700,4.557394,0.263200,0.069400,0.230900,0.230900,13.204800,0.027600,1.000000,1.071000,25720,24015


Cluster [0, 1, 4] training size (998, 14)
TRAINING CLUSTER SET [0, 1, 4] FOR EPOCHS5
LOADING MODEL Salesforce/codet5-base-multi-sum
cuda


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len,Bleu,Brevity Penalty,Length Ratio,Translation Length,Reference Length
1,5.728100,5.257356,0.154000,0.025300,0.139400,0.139500,9.917600,0.009300,0.763300,0.787300,18907,24015
2,5.078300,4.824218,0.228400,0.055300,0.205900,0.205800,13.033200,0.024400,1.000000,1.058400,25418,24015
3,4.736300,4.696280,0.241700,0.062300,0.216600,0.216700,13.051600,0.027900,1.000000,1.053700,25305,24015
4,4.562800,4.666374,0.251500,0.063400,0.222800,0.222900,12.946000,0.028700,1.000000,1.043000,25047,24015
5,4.459200,4.660231,0.253400,0.065300,0.225300,0.225500,12.931200,0.028900,1.000000,1.042000,25024,24015


In [15]:
### ENSEMBLE COMPUTE
test_result_df = ensemble_compute(test_result_df=test_result_df,
                                  optimal_ensemble_map=optimal_ensemble_map)

########## ROUGE PER SETTING

print("Mean")
print(test_result_df.groupby("model_set")["rouge"].mean())

print("STD")
print(test_result_df.groupby("model_set")["rouge"].std())

print(test_result_df.opt_es_id.value_counts())

with open(f'reports/results/{experiment_config["ANALYSIS_POSTFIX"]}/test_results_full.pickle', 'wb') as handle:
    pickle.dump(test_result_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

Mean
model_set
0                    0.128119
1                    0.276487
2                    0.284958
5                    0.288171
10                   0.290581
cluster_[0, 1, 4]    0.253683
cluster_[1]          0.158525
cluster_[3]          0.265418
cluster_[4]          0.228561
ensemble             0.288960
Name: rouge, dtype: float64
STD
model_set
0                    0.137439
1                    0.165795
2                    0.167190
5                    0.168888
10                   0.166432
cluster_[0, 1, 4]    0.167487
cluster_[1]          0.150029
cluster_[3]          0.168580
cluster_[4]          0.166655
ensemble             0.168918
Name: rouge, dtype: float64
opt_es_id
5              21410
10              2130
2                840
1                390
cluster_[3]      130
cluster_[4]      100
Name: count, dtype: int64


In [16]:
test_result_df.opt_es_id.value_counts()

opt_es_id
5              21410
10              2130
2                840
1                390
cluster_[3]      130
cluster_[4]      100
Name: count, dtype: int64